In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import re
from datetime import date




In [2]:
Lenovo_url = "https://www.zap.co.il/models.aspx?sog=c-pclaptop&db239046=239063&pageinfo=12"
#Asus_url = "https://www.zap.co.il/models.aspx?sog=c-pclaptop&db239046=239052"
#HP_url = "https://www.zap.co.il/models.aspx?sog=c-pclaptop&db239046=239061&pageinfo=8&q=%D7%9E%D7%97%D7%A9%D7%91%D7%99%D7%9D%20%D7%A0%D7%99%D7%99%D7%93%D7%99%D7%9D"
#Dell_url = "https://www.zap.co.il/models.aspx?sog=c-pclaptop&db239046=239055&pageinfo=4"
#Apple_url = "https://www.zap.co.il/models.aspx?sog=c-pclaptop&db239046=239051&pageinfo=3"

In [3]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(Lenovo_url)


In [4]:
manufacturer = []
gaming_adapt = []
operating_system = []
weight = []
RAM_memory_volume = []
CPU_speed = []
processor_model = []
processor_generation=[]
memory_type=[]
storage_capacity = []
hard_drive_type = []
optical_drive =[]
screen_size = []
screen_resolution = []
screen_type = []
display_refresh_rate = []
touch_screen = []
webcam=[]
security_measures=[]
cellular_modem=[]
low_price_temp=[]
low_price=[]
high_price_temp = []
high_price = []
average_price_temp = []
average_price = []
models_title=[]
Data_acquisition_date=[]
comp_model_url=[]
computer_entry_date=[]

In [5]:
def nextPage(driver, current_page):
    print("arrived to next page function")
    driver.get(current_page)
    time.sleep(4)
    
    try:
        element = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.page-item.next")))
        
        # Click on the element
        element.click()
        print("moved to url:", driver.current_url)
        
    except:
        try:
            # Find the selected element
            print("new nextpage function started")

            selected_element = driver.find_element(By.CSS_SELECTOR, '.page-item.selected')

            # Find the next sibling element
            next_element = selected_element.find_element(By.XPATH, 'following-sibling::a')

            # Get the href attribute value of the next element
            href = next_element.get_attribute('href')
            print("moved to url:", href)
            print("new nextpage function worked")
        except:
            print("No next element found or selected element not found.")

In [6]:
def computers_url_in_page(driver):
    try:
        models_url_list = []
        # Wait for the elements to be present and clickable
        wait = WebDriverWait(driver, 10)
        elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "ModelPic")))

        # Iterate over the elements
        for element in elements:
            try:
                # Perform actions on each element
                models_url_list.append(element.get_attribute('href'))

            except Exception as e:
                models_url_list.append(" ")
                print(f"Error occurred while performing actions on an element: {str(e)}")

    except Exception as e:
        print(f"couldnt get the list")
    
#    print("page is:",driver.current_url, ", found:", len(models_url_list), "computers, list of urls:", models_url_list)
    
    return (models_url_list)
 

In [7]:
#get price

def prices(driver):
    
    low_price_temp.clear()
    high_price_temp.clear()
    average_price_temp.clear()
    
    
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    price_wrappers = soup.find_all(class_="price-wrapper")
    
    # Filter out elements with class names "price-wrapper product final" and "price-wrapper product total-no-shipping"
    #price_wrappers = [item for item in price_wrappers if "price-wrapper product final" not in item.get('class', []) and "price-wrapper product total-no-shipping" not in item.get('class', [])]


    # Extract the text from each element
    prices = [wrapper.get_text(strip=True) for wrapper in price_wrappers]

    flag=0     
    split_prices = []
    #cleaning the text to minPrice highPrice and average
    for price in prices:
        try:
            #if I get range 
            numbers = re.findall(r'\d{1,3}(?:,\d{3})*(?:\.\d+)?', price)
            min_price = int(numbers[1].replace(',', ''))
            max_price = int(numbers[0].replace(',', ''))
            average = (min_price + max_price) / 2
            split_prices.append({
                'minPrice': min_price,
                'maxPrice': max_price,
                'average': average
            })
            
        except:
            if(flag%3==0):
                clean_price = re.sub(r'[^\d]', '', price) #delete signs such as ₪ or $
                split_prices.append({
                    'minPrice': " ",
                    'maxPrice': " ",
                    'average': int(clean_price.replace(',', ''))
                })
            flag+=1

            

    for j in range(len(split_prices)):

        low_price_temp.append(split_prices[j]['minPrice'])
        high_price_temp.append(split_prices[j]['maxPrice'])
        average_price_temp.append(split_prices[j]['average'])




In [8]:
def models(driver):
    html = driver.page_source
    time.sleep(2)
    soup = BeautifulSoup(html, 'html.parser')
    h1_tag = soup.find('div', class_='model-page-details').find('h1')

    if h1_tag:
        h1_text = h1_tag.text.strip()
        print(h1_text)
        models_title.append(h1_text)
    else:
        print("No <h1> tag found.")
        models_title.append(" ")

    #models_title = [re.sub(r'[\u0590-\u05FF]+', '', name) for name in models_title]



In [9]:
def print_data():
    print("manufacturer=", len(manufacturer), ",", manufacturer)
    print("gaming_adap=:", len(gaming_adapt), ",", gaming_adapt)
    print("operating_system=", len(operating_system), ",", operating_system)
    print("weight=", len(weight), ",", weight)
    print("RAM_memory_volume=", len(RAM_memory_volume), ",", RAM_memory_volume)
    print("CPU_speed=", len(CPU_speed), ",", CPU_speed)
    print("processor_model=", len(processor_model), ",", processor_model)
    print("processor_generation=", len(processor_generation), ",", processor_generation)
    print("memory_type=", len(memory_type), ",", memory_type)
    print("storage_capacity=", len(storage_capacity), ",", storage_capacity)
    print("hard_drive_type=", len(hard_drive_type), ",", hard_drive_type)
    print("optical_drive=", len(optical_drive), ",", optical_drive)
    print("screen_size=", len(screen_size), ",", screen_size)
    print("screen_resolution=", len(screen_resolution), ",", screen_resolution)
    print("screen_type=", len(screen_type), ",", screen_type)
    print("display_refresh_rate=", len(display_refresh_rate), ",", display_refresh_rate)
    print("touch_screen=", len(touch_screen), ",", touch_screen)
    print("webcam=", len(webcam), ",", webcam)
    print("security_measures=", len(security_measures), ",", security_measures)
    print("cellular_modem=", len(cellular_modem), ",", cellular_modem)
    print("low_price=", len(low_price), ",", low_price)
    print("high_price=", len(high_price), ",", high_price)
    print("average_price=", len(average_price), ",", average_price)
    print("models=", len(models_title), ",", models_title)
    print("Data_acquisition_date=", len(Data_acquisition_date), ",", Data_acquisition_date)
    print("comp_model_url=", len(comp_model_url), ",", comp_model_url)
    print("computer_entry_date=", len(computer_entry_date), ",", computer_entry_date)


In [10]:
def pages_number(driver):

    time.sleep(2)
    wait = WebDriverWait(driver, 10)
    last_page_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@aria-label='אחרון']")))
    driver.execute_script("arguments[0].click();", last_page_button)
    element = wait.until(EC.presence_of_element_located((By.XPATH, "//a[contains(@class, 'page-item selected')]")))
    last_page_number = element.text

    number_of_pages= int(last_page_number)

     #get back to the first page
    driver.get(HP_url)
    
    return(last_page_number)


In [11]:
wait = WebDriverWait(driver, 15)
couldnt_get_specifications=[]
#for i in range( ( pages_number(driver) ) -1):
for i in range(10):

    current_page = driver.current_url
    computers_urls = computers_url_in_page(driver)
    print("computers_urls:", computers_urls)
    
    #get price
    extract_price = prices(driver)
    print("average: " , average_price_temp, "length:", len(average_price_temp))
    
    counter_deleted_price = 0
    
    print("moving page ..")
    print(f"page number : {i+1}")

    
    for j in range(len(computers_urls)):
#    for j in range(1):
        print(f"computer number {j+1} in this page")
    
        try:
#trying to skip the click on 'specifications' button by changing the url to the 'specifications' page
            if "model.aspx" not in computers_urls[j]:
                print("url:", computers_urls[j], "does not contain 'model.aspx'")
            
                #delete the price of the computer that couldnt be reached
                low_price_temp.pop(j - counter_deleted_price)
                high_price_temp.pop(j - counter_deleted_price)
                average_price_temp.pop(j - counter_deleted_price)
                counter_deleted_price+=1
            
                continue  # Could not get relevant information, move to the next iteration
           
            
            else:
                computers_urls[j] = computers_urls[j].replace('model.aspx', 'compmodels.aspx')
                driver.get(computers_urls[j])
                time.sleep(4)
 
        except:
            driver.get(computers_urls[j])
            time.sleep(2)

            try:
                specifications = wait.until(EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, "מפרט טכני מלא")))
                if(specifications):
                    driver.execute_script("arguments[0].click();", specifications)
                    print("clicked on specifications")
                else:
                     raise Exception("couldnt reach Specifications")

            except:
                couldnt_get_specifications.append(computers_urls[j])
                print('could not click on', driver.title, "url:", computers_urls[j])
            
                #delete the price of the computer that couldnt be reached
                low_price_temp.pop(j - counter_deleted_price)
                high_price_temp.pop(j - counter_deleted_price)
                average_price_temp.pop(j - counter_deleted_price)
                counter_deleted_price+=1
                
            
                continue  # Could not get relevant information, move to the next iteration

        try:
            
            time.sleep(2)
            computer_model = models(driver)    #get model
            
        except:
            
            #in case the url leads to the actual store and not to Zap page
            print("got non desired page:", computers_urls[j])
            
            #delete the price of the computer that couldnt be reached
            low_price_temp.pop(j - counter_deleted_price)
            high_price_temp.pop(j - counter_deleted_price)
            average_price_temp.pop(j - counter_deleted_price)
            counter_deleted_price+=1
               
            #skip to the next iteration
            continue
            
            
        time.sleep(15)        
        specifications_details = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "ParamColValue")))
        values = [element.text.strip() for element in specifications_details]
        print("current url is:", driver.current_url)
        print ("computer values:", values)


        #add to the relevant arr
#        manufacturer.append(values[0])
 #       computer_entry_date.append(values[1])
  #      gaming_adapt.append(values[2])
   #     operating_system.append(values[3])
   #     weight.append(values[5])
    #    RAM_memory_volume.append(values[8])
     #   CPU_speed.append(values[9])
      #  processor_model.append(values[10])
       # processor_generation.append(values[11])
        #memory_type.append(values[12])
#        storage_capacity.append(values[13])
 #       hard_drive_type.append(values[14])
  #      optical_drive.append(values[15])
   #     screen_size.append(values[16])
    #    screen_resolution.append(values[17])
     #   screen_type.append(values[18])
      #  display_refresh_rate.append(values[19])
       # touch_screen.append(values[20])
 #       webcam.append(values[22])
  #      security_measures.append(values[23])
   #     cellular_modem.append(values[26])
    #    Data_acquisition_date.append(date.today()) #today's date
     #   comp_model_url.append(computers_urls[j])
        
        
        
        manufacturer.append(values[0])
        computer_entry_date.append(values[4])
        gaming_adapt.append(values[5])
        operating_system.append(values[6])
        weight.append(values[8])
        RAM_memory_volume.append(values[11])
        CPU_speed.append(values[12])
        processor_model.append(values[13])
        processor_generation.append(values[14])
        memory_type.append(values[15])
        storage_capacity.append(values[16])
        hard_drive_type.append(values[17])
        optical_drive.append(values[18])
        screen_size.append(values[19])
        screen_resolution.append(values[20])
        screen_type.append(values[21])
        display_refresh_rate.append(values[22])
        touch_screen.append(values[23])
        webcam.append(values[25])
        security_measures.append(values[26])
        cellular_modem.append(values[29])
        Data_acquisition_date.append(date.today()) #today's date
        comp_model_url.append(computers_urls[j])


    
    #arrange the models array
    filter_models = [re.sub(r'[\u0590-\u05FF]+', '', name) for name in models_title]
    models_title = filter_models
 

    #add the prices of all the computers that was got successfully
    low_price = low_price + low_price_temp
    high_price = high_price + high_price_temp
    average_price = average_price + average_price_temp

        
    
    
    print('page number', i+1, 'data:')
    print_data()
    
    print("--------------------------------------------------------------------------------")
    print("--------------------------------------------------------------------------------")
    print("--------------------------------------------------------------------------------")
    print("--------------------------------------------------------------------------------")
    print("--------------------------------------------------------------------------------")
    #moving to the next page
    nextPage(driver, current_page)
    #driver.quit()
            


computers_urls: ['https://www.zap.co.il/model.aspx?modelid=1167682', 'https://www.zap.co.il/model.aspx?modelid=1177445', 'https://www.zap.co.il/model.aspx?modelid=1141064', 'https://www.zap.co.il/model.aspx?modelid=1189531', 'https://www.zap.co.il/model.aspx?modelid=1152196', 'https://www.zap.co.il/model.aspx?modelid=1164743', 'https://www.zap.co.il/model.aspx?modelid=1167795', 'https://www.zap.co.il/model.aspx?modelid=1111307', 'https://www.zap.co.il/model.aspx?modelid=1165584', 'https://www.zap.co.il/model.aspx?modelid=1171586', 'https://www.zap.co.il/model.aspx?modelid=1169789', 'https://www.zap.co.il/model.aspx?modelid=1121302', 'https://www.zap.co.il/model.aspx?modelid=1161897', 'https://www.zap.co.il/model.aspx?modelid=1167030', 'https://www.zap.co.il/model.aspx?modelid=1170363', 'https://www.zap.co.il/model.aspx?modelid=1161886', 'https://www.zap.co.il/model.aspx?modelid=1179110', 'https://www.zap.co.il/model.aspx?modelid=1192416', 'https://www.zap.co.il/model.aspx?modelid=11962

MaxRetryError: HTTPConnectionPool(host='localhost', port=58012): Max retries exceeded with url: /session/ea4532642371e645a7fd40667f08e399/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa7886e93a0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
df = pd.DataFrame(
    {"model":models_title, 
     "manufacturer": manufacturer, 
     "gaming_adapt": gaming_adapt, 
     "operating_system": operating_system, 
     "weight": weight, 
     "RAM_memory_volume": RAM_memory_volume,
     "CPU_speed": CPU_speed,
     "processor_model": processor_model, 
     "processor_generation": processor_generation, 
     "memory_type": memory_type,
     "storage_capacity": storage_capacity, 
     "hard_drive_type": hard_drive_type,
     "optical_drive": optical_drive, 
     "screen_size": screen_size,
     "screen_resolution": screen_resolution,
     "screen_type": screen_type,
     "display_refresh_rate": display_refresh_rate,
     "touch_screen": touch_screen,
     "webcam": webcam,
     "security_measures": security_measures, 
     "cellular_modem": cellular_modem, 
     "low_price": low_price, 
     "high_price": high_price, 
     "average_price":average_price,
     "Data_acquisition_date":Data_acquisition_date,
     "comp_model_url":comp_model_url,
     "computer_entry_date":computer_entry_date
     
    })

print(df)

df.to_csv(r"Lenovo.csv", mode='a', index=True)  

